In [2]:
import os
import sys
sys.path.append('/dfs/scratch1/simran/tutorial/contextual-embeddings/')
import numpy as np
import pickle
import random
import ujson
import argparse
import pandas as pd
import pickle
import json
import csv
from collections import defaultdict, OrderedDict
import jsonlines

/dfs/scratch1/simran/tutorial/bootleg-internal/.bootinternal/lib/python3.6/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
/dfs/scratch1/simran/tutorial/bootleg-internal/.bootinternal/lib/python3.6/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


# preliminary loading

In [3]:
def load_mentions(file): 
    lines = []
    with jsonlines.open(file) as f: 
        for line in f: 
            new_line = {
                'id': line['id'],
                'sentence': line['sentence'],
                'aliases': line['aliases'], 
                'spans': line['spans'],
                'qids': line['qids'],
                'anchor': line['anchor'],
                'sent_idx_unq': line['sent_idx_unq'],
                'probs': line['probs'],
                'ctx_emb_ids': line['ctx_emb_ids'],
                'entity_ids': line['entity_ids']
            }
            lines.append(new_line)
    return pd.DataFrame(lines)

In [4]:
def load_dataset(file):
    lines = []
    with json.open(file) as f: 
        lines.append(new_line)
    return pd.DataFrame(lines)

In [36]:
directory = '/dfs/scratch1/simran/tacred/tacred-relation-bootleg/dataset_bootleg_cidr_model/bootleg_09042020/091120_augmt_pronoun_1person/'
boot_labels_file = "{}bootleg_labels.jsonl".format(directory)
bootleg_labels_df = load_mentions(boot_labels_file)

# generate a remapping for the static entit ids

In [29]:
static_ent_ids_remap = []

for ind, row in bootleg_labels_df.iterrows():
    static_ids = row['entity_ids']
    for idx in static_ids:
        static_ent_ids_remap.append(idx)

static_ent_ids_remap.append(-1) # unknown
        

In [30]:
print(len(static_ent_ids_remap))
print(len(set(static_ent_ids_remap)))

675465
64055


In [31]:
remap_static_ent_id_dict = {}
count = 0
for idx in set(static_ent_ids_remap):
    remap_static_ent_id_dict[count] = idx
    count += 1

# extract out the static entity embs based on the remapping

In [32]:
datafile = '/dfs/scratch1/simran/tacred/tacred-relation-bootleg/dataset_bootleg_cidr_model/bootleg_downloads/static_embeddings.npy'
dir = os.path.dirname(datafile)
emb_data = np.load(datafile)

In [33]:
result_array = np.empty((len(set(static_ent_ids_remap)), 200))
from tqdm import tqdm

idx_lst = []
for idx_new, idx_orig in tqdm(remap_static_ent_id_dict.items()):
    idx_lst.append(idx_orig)
result_array = emb_data[np.array(idx_lst)] 

100%|██████████| 64055/64055 [00:00<00:00, 894620.42it/s]


In [35]:
np.save('{DIR}/static_ent_emb_tacred.npy'.format(DIR=dir), result_array)